In [2]:
N, T, F = 100, 20, 11 # n_samples, sequence lenght, number of features

Generer les données dvec plus d'aleatoire


In [4]:
import os
import numpy as np
from string import ascii_lowercase
from pathlib import Path
import json
import scipy.stats as stats
import scipy.special as special

sandbox = [
    lambda: float(stats.poisson.rvs(mu=np.random.uniform(100, 300))),
    lambda: float(stats.nbinom.rvs(n=np.random.uniform(10, 50), 
                                    p=np.random.uniform(0.1, 0.9))),
    lambda: float(stats.randint.rvs(low=100, high=800)),
    lambda: float(np.floor(stats.norm.rvs(loc=np.random.uniform(100, 500), 
                                            scale=np.random.uniform(50, 200)))),
    lambda: float(np.ceil(stats.uniform.rvs(loc=100, scale=700))),
    lambda: float(np.abs(int(stats.norm.rvs(loc=300, scale=100)) * 
                        int(stats.uniform.rvs(loc=1, scale=2)))),
    lambda: float(np.floor((stats.norm.rvs(loc=200, scale=50) / 
                            stats.norm.rvs(loc=100, scale=25)) * 100)),
    lambda: float(np.random.randint(100, 800) % float(np.random.uniform(10, 50)) + 100),
    lambda: float(stats.binom.rvs(n=500, p=np.random.uniform(0.1, 0.5))),
    lambda: float(stats.geom.rvs(p=np.random.uniform(0.1, 0.5))),
    lambda: float(np.power(np.random.randint(2, 10), 
                            np.random.randint(1, 3)) % 700 + 100),
    lambda: float(np.abs(np.sin(np.random.uniform(0, np.pi)) * 500 + 300)),
    lambda: float(np.random.choice([
        101, 103, 107, 109, 113, 127, 131, 137, 139, 
        149, 151, 157, 163, 167, 173, 179, 181, 191, 
        193, 197, 199
    ]) + np.random.randint(-50, 50)),
    lambda: float(np.log(np.random.randint(10, 1000)) * 50 + 100),
    lambda: float(np.abs(np.random.normal(0, 1) ** np.random.uniform(1, 3)) * 100 % 700 + 100),
    lambda: float(special.factorial(np.random.randint(3, 6)) % 700 + 100),
    lambda: float(np.gcd(np.random.randint(100, 800), np.random.randint(100, 800))),
    lambda: float(np.lcm(np.random.randint(10, 100), np.random.randint(10, 100)) % 700 + 100)
]


letter_strategies = {
    letter: np.random.choice(sandbox, size=F, replace=False).tolist()
    for letter in ascii_lowercase
}

def generate_advanced_timeseries(N=N, T=T, F=F, distribution_strategies = letter_strategies):
    base_dir = f"{N} {T} {F}"
    os.makedirs(base_dir, exist_ok=True)
    data = []

    for letter, strategies in distribution_strategies.items():
        letter_file = Path(base_dir, f"letter_{letter}.json")
        letter_data = []

        for _ in range(N):
            sample = [
                [int(max(100, min(800, strategy()))) for strategy in strategies]
                for _ in range(T)
            ]
            letter_data.append(sample)

        data.append(letter_data)
        letter_file.write_text(json.dumps(letter_data))

    dataset = Path(base_dir, f"dataset.json")
    dataset.write_text(json.dumps(data))
    
    return data

# Generate the advanced dataset
generated_data = generate_advanced_timeseries()

In [6]:
data_path = Path(f"{N} {T} {F}")
dataset = {"x": [], "y":[]}
for letter in ascii_lowercase:
    letter_file = data_path / f"letter_{letter}.json"
    letter_data = json.loads(letter_file.read_text())
    dataset["x"].extend(letter_data)
    dataset["y"].extend([letter for i in range(N)])

dataset["x"][0], dataset["y"][0]        

([[100, 118, 759, 100, 245, 405, 298, 447, 100, 240, 742],
  [100, 102, 756, 108, 100, 362, 210, 392, 100, 196, 546],
  [100, 203, 718, 100, 188, 366, 298, 425, 100, 268, 446],
  [100, 206, 202, 201, 104, 414, 421, 297, 100, 271, 436],
  [100, 100, 777, 176, 158, 402, 268, 232, 100, 203, 506],
  [100, 140, 297, 100, 298, 426, 592, 708, 100, 117, 328],
  [100, 109, 561, 133, 125, 402, 488, 124, 100, 271, 196],
  [100, 206, 338, 119, 448, 399, 226, 262, 100, 296, 732],
  [100, 107, 707, 100, 129, 361, 172, 550, 100, 266, 414],
  [100, 164, 167, 100, 245, 397, 330, 614, 100, 151, 736],
  [100, 134, 721, 100, 444, 399, 190, 390, 100, 205, 217],
  [100, 112, 613, 100, 529, 380, 167, 506, 100, 268, 199],
  [100, 184, 389, 100, 312, 393, 483, 299, 100, 108, 800],
  [100, 116, 752, 100, 614, 429, 390, 594, 100, 100, 157],
  [100, 100, 282, 118, 112, 440, 157, 308, 100, 224, 309],
  [100, 197, 711, 100, 101, 431, 456, 190, 100, 186, 165],
  [100, 100, 133, 100, 148, 440, 454, 706, 100, 269, 310

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Convert data and labels to numpy arrays
X = np.array(dataset["x"])
y = np.array(dataset["y"])

# One-hot encode labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_one_hot = tf.keras.utils.to_categorical(y_encoded)

# Normalize the data
scaler = StandardScaler()
X_normalized = np.zeros_like(X, dtype=float)
for i in range(X.shape[0]):
    X_normalized[i] = scaler.fit_transform(X[i])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y_one_hot, test_size=0.2, random_state=42
)

# Build the model
def create_time_series_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        # LSTM layers for sequence processing
        tf.keras.layers.LSTM(64, input_shape=input_shape, return_sequences=True),
        tf.keras.layers.LSTM(32),
        
        # Dense layers for classification
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        
        # Output layer
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Get input shape
input_shape = (X_train.shape[1], X_train.shape[2])
num_classes = y_one_hot.shape[1]

# Create and train the model
model = create_time_series_model(input_shape, num_classes)

# Train the model
history = model.fit(
    X_train, y_train, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    verbose=1
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")

# Optional: Make predictions
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

print("\nPrediction Details:")
for pred, true in zip(predicted_classes, true_classes):
    print(f"Predicted: {label_encoder.inverse_transform([pred])[0]}, "
          f"True: {label_encoder.inverse_transform([true])[0]}")


In [61]:
model.save("mode.h5")

In [25]:
X[0].shape

(20, 11)

In [50]:
_model = tf.keras.models.load_model(f"{N} {T} {F}/[0] model.keras")
pred = _model.predict(X_test[0:1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step


In [56]:
import tensorflow as tf
import json
import numpy as np

# Assuming 'model' is your trained TensorFlow model from the previous script

# Export weights to JSON
def export_weights_to_json(model, filename='model.weights.json'):
    weights = {}
    for layer in model.layers:
        layer_weights = layer.get_weights()
        if layer_weights:  # Only store layers with weights
            weights[layer.name] = [w.tolist() for w in layer_weights]
    
    with open(filename, 'w') as f:
        json.dump(weights, f)
    print(f"Weights exported to {filename}")

# Export model architecture and weights
def export_full_model(model, architecture_filename='model_architecture.json', 
                      weights_filename='model.weights.h5'):
    # Save model architecture as JSON
    model_json = model.to_json()
    with open(architecture_filename, 'w') as f:
        f.write(model_json)
    
    # Save weights in HDF5 format
    model.save_weights(weights_filename)
    print(f"Model architecture exported to {architecture_filename}")
    print(f"Model weights exported to {weights_filename}")

# TensorFlow.js export
def export_to_tfjs(model, output_dir='tfjs_model'):
    import tensorflowjs as tfjs
    tfjs.converters.save_keras_model(model, output_dir)
    print(f"Model exported to TensorFlow.js format in {output_dir}")

# Demonstrate usage
export_weights_to_json(model)
export_full_model(model)
export_to_tfjs(model)

# Loading methods

# Load weights from JSON
def load_weights_from_json(model, filename='model.weights.json'):
    with open(filename, 'r') as f:
        weights_dict = json.load(f)
    
    for layer in model.layers:
        if layer.name in weights_dict:
            # Convert back to numpy arrays
            layer_weights = [np.array(w) for w in weights_dict[layer.name]]
            layer.set_weights(layer_weights)
    
    return model

# Load TensorFlow model
def load_full_model(architecture_filename='model_architecture.json', 
                    weights_filename='model.weights.h5'):
    # Load model architecture
    with open(architecture_filename, 'r') as f:
        model_json = f.read()
    
    loaded_model = tf.keras.models.model_from_json(model_json)
    
    # Load weights
    loaded_model.load_weights(weights_filename)
    
    return loaded_model

# TensorFlow.js loading example (in JavaScript)
"""
// JavaScript code for loading TensorFlow.js model
import * as tf from '@tensorflow/tfjs';

async function loadModel() {
    const model = await tf.loadLayersModel('path/to/tfjs_model/model.json');
    return model;
}
"""

Weights exported to model.weights.json
Model architecture exported to model_architecture.json
Model weights exported to model.weights.h5


/workspaces/harmony-gloves/venv/lib/python3.12/site-packages/tensorflowjs/read_weights.py:28: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.uint8, np.uint16, np.object, np.bool]


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations